# Init

In [59]:
import sys
import os
import io
import shutil
import logging, logging.config, yaml
from logging.handlers import RotatingFileHandler
from logging import Formatter, FileHandler, StreamHandler
import json
import pandas as pd
import numpy as np
import string
from io import StringIO
import random
import math
import time
import re
from datetime import datetime
from datetime import date
from datetime import timedelta
from datetime import timezone
from pytz import timezone
import pytz
import pause
import humanfriendly
from bs4 import BeautifulSoup
from tabulate import tabulate
from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from flask import Flask, session, render_template, request, jsonify, redirect, g, url_for
import requests
working_time_zone='Europe/Kiev'
alpari_date_format="%m/%d/%Y"
fh_time_format="%Y-%m-%d %H:%M:%S"
fh_date_format='%Y-%m-%d'
os.environ['TZ'] = working_time_zone
time.tzset()
work_dir=os.getcwd()
if 'nginx-flask-plumber' != os.path.basename(work_dir):
    os.chdir(os.path.dirname(work_dir))
work_dir=os.getcwd()
print(work_dir)

/home/jovyan/work/R/nginx-flask-plumber


In [60]:
app = Flask(__name__)
app.debug = True
log_file=os.path.join(work_dir, os.path.basename(work_dir) + '.log')
print(log_file)
open(log_file, 'w').close()
os.chmod(log_file, 0o665)
formatter = Formatter('%(asctime)s:%(levelname)s:%(module)s:%(funcName)s:%(lineno)d:%(message)s', 
                              datefmt=fh_time_format)
handler = RotatingFileHandler(log_file, maxBytes=10000, backupCount=1)
handler.setLevel(logging.INFO)
handler.setFormatter(formatter)
app.logger.addHandler(handler)

/home/jovyan/work/R/nginx-flask-plumber/nginx-flask-plumber.log


# Flask routes

In [61]:
# Main page
# curl -i http://localhost:80/pnews/
@app.route('/')
def flask_index():
    app.logger.info("main page opened.")
    return "Service pnews (language python) is ready!\r\n"
# Health check
# curl -i http://localhost:80/pnews/health
@app.route('/health')
def flask_health():
    app.logger.info("checked.")
    return 'Healthy\r\n'
# Echo
# curl -i http://localhost:80/pnews/echo?msg=hello
@app.route('/echo', methods=['GET'])
def flask_echo(): 
    msg=request.args.get('msg', '')
    app.logger.info(msg)
    return jsonify(msg='The message is: "' + msg + '"\r\n')
# Pause 
# curl -i http://localhost:80/pnews/pause/2
@app.route('/pause/<int:duration>', methods=['GET'])
def flask_pause(duration):
    app.logger.info("Pause " + str(duration) + " seconds")
    dt=datetime.now()+timedelta(seconds=duration)
    pause.until(dt)
    return "Pause of " + str(duration) + ' seconds finished'
@app.route('/browser_session')
def flask_browser_session():
    driver=open_browser()
    close_browser(driver)
    msg="browser session stopped"
    app.logger.info(msg)
    return msg
# @app.route('/open_browser')
# def flask_open_browser():
#     driver=open_browser()
#     session['driver']=driver
#     return 'browser opened see log for text content'
# @app.route('/close_browser')
# def flask_close_browser():
#     driver=session['driver']
#     close_browser(driver)
#     return "closed"
@app.route('/test_connectivity')
def flask_test_connectivity():
    app.logger.info('started')
    msg='finished successfully'
    app.logger.info(msg)
    return msg
@app.route('/text_message', methods=('POST',))
def flask_text_message():
    head=[]
    try:
        json=request.get_json()#.get('thing2')
        df=accept_text_data(json)
        head=df.head()
    except:
        msg='Bad payload'
        app.logger.info(msg)
        return msg
    output = StringIO()
    output.write(head.to_csv(index=False))
    msg=output.getvalue()
    output.close()    
    app.logger.info("\r\n" + msg)
    return msg

# Testing

In [62]:
def create_test_df():
    date_rng = pd.date_range(start='2019-01-24', end='2019-01-25', freq='H')
    l=len(date_rng)
    dts = pd.to_datetime(date_rng, format=fh_time_format)  
    hashes=[''.join(random.choice(string.ascii_uppercase + string.digits) for _ in range(3)) for x in range(l)]
    ns=list(range(l))
    vs=random.sample(range(100), l)
    d={'dt': dts, "event":hashes, "number": ns, "value": vs}
    df=pd.DataFrame(d)
    df['dt']=[dt.strftime('%Y-%m-%d %H:%M:%S') for dt in df.dt]
    return df

In [63]:
def create_test_series():
    date_rng = pd.date_range(start='2019-01-24', end='2019-01-25', freq='H')
    l=len(date_rng)
    dts = pd.to_datetime(date_rng, format=fh_time_format)  
    ns=list(range(l))
    vs=random.sample(range(100), l)
    d={'dt': dts, "number": ns, "value": vs}
    df=pd.DataFrame(d)
    df['dt']=[dt.strftime('%Y-%m-%d %H:%M:%S') for dt in df.dt]
    return df

In [104]:
def create_payload(data_type="sr"):
    if data_type not in ['sr', 'df']:
        raise Exception("Invalid data_type! " + data_type)
        return None
    output = StringIO()
    if data_type == "df":
        df=create_test_df()
        output.write(df.to_csv(index=False))
    if data_type == 'sr':
        sr=create_test_series()
        output.write(sr.to_csv(index=False))
    payload = {"spec":data_type, "text":output.getvalue()}
    output.close()
    return payload
def post_text_data(data_type="sr", url="http://pnews:5000"):
    payload=create_payload(data_type)
    data=json.dumps(payload)
    url_path=os.path.join(url, "text_message")
    headers = {'Content-Type': 'application/json',}
    r = requests.post(url_path, data=data, headers=headers)
    if r.status_code == requests.codes.ok:
        app.logger.info('data of type '+ data_type + ' posted to ' + url_path)
    return r.status_code
def accept_text_data(payload):
    output = StringIO(payload['text'])
    msg=output.getvalue()
    df=pd.read_csv(output)
    output.close()    
    return df

In [91]:
def write_request_json(out_file='test.json'):
    df=create_test_df()
    df.head()
    df.to_json(out_file)
    df1=pd.read_json(out_file)
    df1.head()

In [92]:
def read_response_json(in_file='response.json'):
    df=pd.read_json(in_file)
    df.head()

# Browser automation

In [93]:
def open_browser(sleep_time=10):
    app.logger.info('start opening browser')
    cap = DesiredCapabilities.CHROME
    driver = webdriver.Remote(command_executor='http://selenium-hub:4444/wd/hub',desired_capabilities=cap) 
    url="https://www.google.com"
    app.logger.info('Opening URL: ' + url + ' waiting approximately ' + str(sleep_time) + ' seconds')
    driver.get(url)
    wait = WebDriverWait(driver, sleep_time)
    element = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, '.gLFyf')))    
    app.logger.info('done')
    text=driver.find_element_by_xpath("html").text
    app.logger.info("Resulting text\r\n" + text)
    return driver
def close_browser(driver):
    driver.quit()
    app.logger.info("closed")

# Main

In [ ]:
if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000) # open url http://0.0.0.0:5000/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)


In [17]:
# url="http://pnews:5000"
# r = requests.get(os.path.join(url, 'open_browser'))
# r.status_code == requests.codes.ok

/home/jovyan/work/R/nginx-flask-plumber
/home/jovyan/work/R/nginx-flask-plumber
/home/jovyan/work/R/nginx-flask-plumber/nginx-flask-plumber.log


In [11]:
code=post_text_data('sr')

In [11]:
if r.status_code == requests.codes.ok:
    app.logger.info('data of type '+ data_type + 'posted to ' + url_path)
#  r.text

In [ ]:
# print(os.getcwd())
# file = open('error.html','w') 
# file.write(r.text) 
# file.close() 

In [102]:
data_type="sr"
url="http://pnews:5000"
payload=create_payload(data_type)
data=json.dumps(payload)
url_path=os.path.join(url, "text_message")
headers = {'Content-Type': 'application/json',}
r = requests.post(url_path, data=data, headers=headers)

In [108]:
post_text_data('df', 'http://rnews:5000/text_message')

404

In [44]:
accept_text_data()

,"""index",x,y
0,", 2019-01-31 12:13:30",-1.415955,0.864118
1,", 2019-01-31 12:13:31",-1.401794,-0.704950
2,", 2019-01-31 12:13:32",-1.435208,0.916722
3,", 2019-01-31 12:13:33",0.164022,-0.414343
4,", 2019-01-31 12:13:34",1.054146,-1.048713
5,", 2019-01-31 12:13:35",-0.706376,-0.816655
6,", 2019-01-31 12:13:36",0.083785,1.028604
7,", 2019-01-31 12:13:37",-0.940142,-1.367651
8,", 2019-01-31 12:13:38",0.748499,-0.531278
9,", 2019-01-31 12:13:39",-3.357793,0.897892
